# Biblioteki

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup

# Zadanie 1

## Wysłanie żądania dostępu do zasobu serwera 

In [ ]:
def extract(ancestor, selector=None, attribute=None, returns_list=False):
    if selector:          
        if returns_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.get_text().strip() for tag in ancestor.select(selector)]

        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()

In [ ]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-1"

# https://www.ceneo.pl/114700014/opinie-1

In [ ]:
selectors = {
                "opinion_id"      : (None, "data-entry-id"),
                "author"          : ("span.user-post__author-name", ),
                "recommendation"  : ("span.user-post__author-recomendation > em", ),
                "rating"          : ("span.user-post__score-count", ),
                "content"         : ("div.user-post__text", ),
                "pros"            : ("div.review-feature__title--positives~.review-feature__item", None, True),
                "cons"            : ("div.review-feature__title--negatives~.review-feature__item", None, True),
                "useful"          : ("span[id^='votes-yes']", ),
                "useless"         : ("span[id^='votes-no']", ),
                "post_date"       : ("span.user-post__published > time:nth-child(1)", "datetime"),
                "purchase_date"   : ("span.user-post__published > time:nth-child(2)", "datetime"),
            }

In [ ]:
all_opinions = []

while (url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    #print(type(page_dom))

    opinions = page_dom.select("div.js_product-review")
    #print(type(opinions))

    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }        
        all_opinions.append(single_opinion)
    try: 
        url = "https://www.ceneo.pl" + extract(page_dom, "a.pagination__next", "href")
    except TypeError:
        url = False

## Ekstrakcja składowych pojedynczej opinii 

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation>em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|.review-feature__title--positives~.review-feature__item|pros|
|lista wad|.review-feature__title--negatives~.review-feature__item|cons|
|dla ilu przydatna|[id^="votes-yes"]|useful|
|dla ilu nieprzydatna|[id^="votes-no"]|useless|
|data wystawienia|.user-post__published > time:nth-child(1)["datetime"]|post_date|
|data zakupu|.user-post__published > time:nth-child(2)["datetime"]|purchase_date|

In [ ]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")

with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jfile:
    json.dump(all_opinions, jfile, indent=6, ensure_ascii=False)

In [ ]:
len(all_opinions)